# AdaGrad: Adaptive Learning Rates

## 🎯 What This Notebook Covers

**AdaGrad** (Adaptive Gradient) adapts the learning rate for each parameter individually. In this notebook, we explore:

1. ✅ **Motivation** - Why adapt learning rates per parameter?
2. ✅ **Mathematical Formulation** - How AdaGrad works
3. ✅ **Implementation** - AdaGrad from scratch
4. ✅ **Advantages** - When AdaGrad shines
5. ✅ **Limitations** - The diminishing learning rate problem

### Why This Matters

- **Parameter-Specific Learning**: Different parameters need different learning rates 🎯
- **Sparse Features**: Excellent for sparse data (NLP, recommender systems) 📊
- **Automatic Adaptation**: No manual tuning needed ⚙️

Let's master AdaGrad! 🚀

---

## 1. Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from IPython.display import display, Markdown

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")

## 2. Motivation: Why Adaptive Learning Rates?

### The Problem with Fixed Learning Rates

Consider a neural network with many parameters:

```
Parameter 1: Updated frequently (large gradients)
    → Needs SMALL learning rate
    
Parameter 2: Updated rarely (small gradients)
    → Needs LARGE learning rate
    
Fixed LR: Same for all parameters ❌
    → Suboptimal for both!
```

### AdaGrad's Solution

**Idea**: Adapt learning rate based on historical gradients

- **Frequent updates** → Accumulate large gradient sum → **Smaller effective LR**
- **Rare updates** → Accumulate small gradient sum → **Larger effective LR**

### Real-World Example: Sparse Features

In NLP or recommender systems:
- Common words ("the", "a") → Updated often → Need small LR
- Rare words ("antidisestablishmentarianism") → Updated rarely → Need large LR

AdaGrad handles this automatically! ✨

---

## 3. Mathematical Formulation

### AdaGrad Update Rule

$$
\begin{align}
G_t &= G_{t-1} + g_t^2 \quad \text{(accumulate squared gradients)} \\
\theta_{t+1} &= \theta_t - \frac{\alpha}{\sqrt{G_t + \epsilon}} \cdot g_t \quad \text{(parameter update)}
\end{align}
$$

Where:
- $g_t = \nabla L(\theta_t)$ = gradient at time $t$
- $G_t$ = sum of squared gradients up to time $t$
- $\alpha$ = initial learning rate (e.g., 0.01)
- $\epsilon$ = small constant for numerical stability (e.g., $10^{-8}$)

### Key Insight

The effective learning rate for each parameter is:

$$
\alpha_{\text{effective}} = \frac{\alpha}{\sqrt{G_t + \epsilon}}
$$

- **Large $G_t$** (frequent updates) → **Small effective LR**
- **Small $G_t$** (rare updates) → **Large effective LR**

### Element-wise Operations

All operations are **element-wise** (per parameter):

$$
\theta_{t+1,i} = \theta_{t,i} - \frac{\alpha}{\sqrt{G_{t,i} + \epsilon}} \cdot g_{t,i}
$$

Each parameter has its own accumulated gradient sum!

---

## 4. Visualize Adaptive Learning Rates

In [ ]:
# Simulate gradient history for two parameters
np.random.seed(42)
iterations = 100

# Parameter 1: Frequent large updates
gradients_param1 = np.random.randn(iterations) * 2.0 + 1.0

# Parameter 2: Rare small updates
gradients_param2 = np.random.randn(iterations) * 0.2

# Compute AdaGrad learning rates
alpha = 0.1
epsilon = 1e-8

G1 = np.zeros(iterations)
G2 = np.zeros(iterations)
lr1 = np.zeros(iterations)
lr2 = np.zeros(iterations)

for t in range(iterations):
    if t == 0:
        G1[t] = gradients_param1[t]**2
        G2[t] = gradients_param2[t]**2
    else:
        G1[t] = G1[t-1] + gradients_param1[t]**2
        G2[t] = G2[t-1] + gradients_param2[t]**2
    
    lr1[t] = alpha / (np.sqrt(G1[t] + epsilon))
    lr2[t] = alpha / (np.sqrt(G2[t] + epsilon))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Accumulated gradients
axes[0].plot(G1, linewidth=2.5, label='Parameter 1 (frequent updates)', color='#FF6B6B')
axes[0].plot(G2, linewidth=2.5, label='Parameter 2 (rare updates)', color='#4ECDC4')
axes[0].set_xlabel('Iteration', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Accumulated Squared Gradients (G)', fontsize=12, fontweight='bold')
axes[0].set_title('Gradient Accumulation', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot 2: Effective learning rates
axes[1].plot(lr1, linewidth=2.5, label='Parameter 1 (decreases fast)', color='#FF6B6B')
axes[1].plot(lr2, linewidth=2.5, label='Parameter 2 (decreases slow)', color='#4ECDC4')
axes[1].set_xlabel('Iteration', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Effective Learning Rate', fontsize=12, fontweight='bold')
axes[1].set_title('Adaptive Learning Rates', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Observations:")
print("  • Parameter 1: Large gradients → Fast accumulation → Small LR")
print("  • Parameter 2: Small gradients → Slow accumulation → Large LR")
print("  • AdaGrad automatically adapts!")

## 5. Generate Dataset

In [ ]:
def generate_spiral_data(n_samples=300, noise=0.1):
    """
    Generate spiral dataset for binary classification.
    
    Returns:
    - X: Features (n_x, m)
    - Y: Labels (1, m)
    """
    np.random.seed(42)
    m = n_samples
    
    # Create spiral
    theta = np.linspace(0, 4*np.pi, m//2)
    r = np.linspace(0.5, 2, m//2)
    
    # Class 0: spiral
    X_class0 = np.vstack([r * np.cos(theta), r * np.sin(theta)])
    Y_class0 = np.zeros((1, m//2))
    
    # Class 1: spiral (rotated)
    X_class1 = np.vstack([r * np.cos(theta + np.pi), r * np.sin(theta + np.pi)])
    Y_class1 = np.ones((1, m//2))
    
    # Combine
    X = np.hstack([X_class0, X_class1])
    Y = np.hstack([Y_class0, Y_class1])
    
    # Add noise
    X += np.random.randn(*X.shape) * noise
    
    # Shuffle
    indices = np.random.permutation(m)
    X = X[:, indices]
    Y = Y[:, indices]
    
    return X, Y

# Generate data
X, Y = generate_spiral_data(n_samples=300, noise=0.1)

print(f"Dataset shape: X={X.shape}, Y={Y.shape}")
print(f"Number of samples: {X.shape[1]}")
print(f"Number of features: {X.shape[0]}")

## 6. Neural Network with AdaGrad

In [ ]:
def sigmoid(z):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

def relu(z):
    """ReLU activation function."""
    return np.maximum(0, z)

def relu_derivative(z):
    """Derivative of ReLU."""
    return (z > 0).astype(float)

print("✅ Activation functions defined!")

In [ ]:
class AdaGrad:
    """
    Neural network with AdaGrad optimizer.
    
    Architecture: Input (2) → Hidden (10, ReLU) → Output (1, Sigmoid)
    """
    
    def __init__(self, n_x=2, n_h=10, n_y=1, learning_rate=0.1, epsilon=1e-8, random_seed=42):
        """
        Initialize neural network with AdaGrad.
        
        Parameters:
        - learning_rate: Initial learning rate (α)
        - epsilon: Small constant for numerical stability
        """
        np.random.seed(random_seed)
        
        self.n_x = n_x
        self.n_h = n_h
        self.n_y = n_y
        self.lr = learning_rate
        self.epsilon = epsilon
        
        # Initialize parameters
        self.W1 = np.random.randn(n_h, n_x) * 0.1
        self.b1 = np.zeros((n_h, 1))
        self.W2 = np.random.randn(n_y, n_h) * 0.1
        self.b2 = np.zeros((n_y, 1))
        
        # Initialize accumulated squared gradients
        self.G_dW1 = np.zeros_like(self.W1)
        self.G_db1 = np.zeros_like(self.b1)
        self.G_dW2 = np.zeros_like(self.W2)
        self.G_db2 = np.zeros_like(self.b2)
        
        # Training history
        self.losses = []
        self.accuracies = []
    
    def forward_propagation(self, X):
        """Forward propagation."""
        Z1 = self.W1 @ X + self.b1
        A1 = relu(Z1)
        Z2 = self.W2 @ A1 + self.b2
        A2 = sigmoid(Z2)
        
        cache = {'Z1': Z1, 'A1': A1, 'Z2': Z2, 'A2': A2}
        return A2, cache
    
    def compute_loss(self, Y, A2):
        """Compute binary cross-entropy loss."""
        m = Y.shape[1]
        loss = -np.mean(Y * np.log(A2 + 1e-8) + (1 - Y) * np.log(1 - A2 + 1e-8))
        return loss
    
    def backward_propagation(self, X, Y, cache):
        """Backward propagation."""
        m = X.shape[1]
        Z1, A1, Z2, A2 = cache['Z1'], cache['A1'], cache['Z2'], cache['A2']
        
        # Backprop
        dZ2 = A2 - Y
        dW2 = (1/m) * (dZ2 @ A1.T)
        db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
        
        dA1 = self.W2.T @ dZ2
        dZ1 = dA1 * relu_derivative(Z1)
        dW1 = (1/m) * (dZ1 @ X.T)
        db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
        
        return dW1, db1, dW2, db2
    
    def update_parameters_adagrad(self, dW1, db1, dW2, db2):
        """
        Update parameters using AdaGrad.
        
        G_t = G_{t-1} + g_t^2
        θ_t = θ_{t-1} - (α / sqrt(G_t + ε)) * g_t
        """
        # Accumulate squared gradients
        self.G_dW1 += dW1**2
        self.G_db1 += db1**2
        self.G_dW2 += dW2**2
        self.G_db2 += db2**2
        
        # Update parameters with adaptive learning rates
        self.W1 -= (self.lr / np.sqrt(self.G_dW1 + self.epsilon)) * dW1
        self.b1 -= (self.lr / np.sqrt(self.G_db1 + self.epsilon)) * db1
        self.W2 -= (self.lr / np.sqrt(self.G_dW2 + self.epsilon)) * dW2
        self.b2 -= (self.lr / np.sqrt(self.G_db2 + self.epsilon)) * db2
    
    def compute_accuracy(self, X, Y):
        """Compute accuracy."""
        A2, _ = self.forward_propagation(X)
        predictions = (A2 > 0.5).astype(int)
        accuracy = np.mean(predictions == Y)
        return accuracy
    
    def fit(self, X, Y, epochs=1000, verbose=False):
        """Train the network with AdaGrad."""
        for epoch in range(epochs):
            # Forward propagation
            A2, cache = self.forward_propagation(X)
            
            # Compute loss
            loss = self.compute_loss(Y, A2)
            self.losses.append(loss)
            
            # Compute accuracy
            accuracy = self.compute_accuracy(X, Y)
            self.accuracies.append(accuracy)
            
            # Backward propagation
            dW1, db1, dW2, db2 = self.backward_propagation(X, Y, cache)
            
            # Update parameters with AdaGrad
            self.update_parameters_adagrad(dW1, db1, dW2, db2)
            
            # Print progress
            if verbose and (epoch + 1) % 200 == 0:
                print(f"Epoch {epoch+1:4d}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")
        
        if verbose:
            print(f"\n✅ Training Complete!")
            print(f"   Final Loss: {self.losses[-1]:.4f}")
            print(f"   Final Accuracy: {self.accuracies[-1]:.4f}")
        
        return self

print("✅ AdaGrad class defined!")

## 7. Comparison: AdaGrad vs SGD vs Momentum

In [ ]:
# We'll need SGD and Momentum classes for comparison
class VanillaSGD:
    """Vanilla SGD for comparison."""
    def __init__(self, n_x=2, n_h=10, n_y=1, learning_rate=0.01, random_seed=42):
        np.random.seed(random_seed)
        self.lr = learning_rate
        self.W1 = np.random.randn(n_h, n_x) * 0.1
        self.b1 = np.zeros((n_h, 1))
        self.W2 = np.random.randn(n_y, n_h) * 0.1
        self.b2 = np.zeros((n_y, 1))
        self.losses = []
        self.accuracies = []
    
    def forward_propagation(self, X):
        Z1 = self.W1 @ X + self.b1
        A1 = relu(Z1)
        Z2 = self.W2 @ A1 + self.b2
        A2 = sigmoid(Z2)
        return A2, {'Z1': Z1, 'A1': A1, 'Z2': Z2, 'A2': A2}
    
    def compute_loss(self, Y, A2):
        return -np.mean(Y * np.log(A2 + 1e-8) + (1 - Y) * np.log(1 - A2 + 1e-8))
    
    def backward_propagation(self, X, Y, cache):
        m = X.shape[1]
        Z1, A1, A2 = cache['Z1'], cache['A1'], cache['A2']
        dZ2 = A2 - Y
        dW2 = (1/m) * (dZ2 @ A1.T)
        db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
        dA1 = self.W2.T @ dZ2
        dZ1 = dA1 * relu_derivative(Z1)
        dW1 = (1/m) * (dZ1 @ X.T)
        db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
        return dW1, db1, dW2, db2
    
    def compute_accuracy(self, X, Y):
        A2, _ = self.forward_propagation(X)
        return np.mean((A2 > 0.5).astype(int) == Y)
    
    def fit(self, X, Y, epochs=1000, verbose=False):
        for epoch in range(epochs):
            A2, cache = self.forward_propagation(X)
            self.losses.append(self.compute_loss(Y, A2))
            self.accuracies.append(self.compute_accuracy(X, Y))
            dW1, db1, dW2, db2 = self.backward_propagation(X, Y, cache)
            self.W1 -= self.lr * dW1
            self.b1 -= self.lr * db1
            self.W2 -= self.lr * dW2
            self.b2 -= self.lr * db2
        return self

print("✅ Comparison classes defined!")

In [ ]:
# Training parameters
epochs = 2000

print("🔬 Training Models...\n")

# 1. Vanilla SGD
print("1️⃣  Training Vanilla SGD...")
model_sgd = VanillaSGD(learning_rate=0.01, random_seed=42)
model_sgd.fit(X, Y, epochs=epochs)
print(f"   Final Loss: {model_sgd.losses[-1]:.4f}")

# 2. AdaGrad
print("\n2️⃣  Training AdaGrad...")
model_adagrad = AdaGrad(learning_rate=0.1, random_seed=42)
model_adagrad.fit(X, Y, epochs=epochs)
print(f"   Final Loss: {model_adagrad.losses[-1]:.4f}")

print("\n✅ All experiments complete!")

## 8. Visualize Results

In [ ]:
# Plot loss curves
plt.figure(figsize=(16, 10))

plt.plot(model_sgd.losses, linewidth=2.5, label='Vanilla SGD', 
        color='#FF6B6B', alpha=0.8)
plt.plot(model_adagrad.losses, linewidth=2.5, label='AdaGrad', 
        color='#4ECDC4', alpha=0.8)

plt.xlabel('Epoch', fontsize=13, fontweight='bold')
plt.ylabel('Loss', fontsize=13, fontweight='bold')
plt.title('Loss Curves: AdaGrad vs Vanilla SGD', fontsize=15, fontweight='bold')
plt.legend(fontsize=12, loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Observations:")
print("  • AdaGrad: Faster initial convergence")
print("  • AdaGrad: Automatic learning rate adaptation")
print("  • AdaGrad: May slow down in later epochs (diminishing LR)")

## 9. The Diminishing Learning Rate Problem

### AdaGrad's Main Limitation

Since $G_t$ only **accumulates** (never decreases):

$$
G_t = G_{t-1} + g_t^2 \implies G_t \text{ grows monotonically}
$$

This means:

$$
\alpha_{\text{effective}} = \frac{\alpha}{\sqrt{G_t + \epsilon}} \to 0 \text{ as } t \to \infty
$$

### Consequences

```
Early Training:
    • G_t is small
    • Effective LR is large
    • ✅ Fast progress

Late Training:
    • G_t is very large
    • Effective LR is tiny
    • ❌ Training stops (premature convergence)
```

### When This is a Problem

- **Long training runs**: LR becomes too small
- **Non-convex problems**: May get stuck in saddle points
- **Deep networks**: Accumulation happens faster

### Solution: RMSProp

RMSProp (next notebook) fixes this by using **exponentially weighted average** instead of sum!

---

## 10. Summary and Key Takeaways

### What We Learned

✅ **Motivation**
- Different parameters need different learning rates
- Sparse features benefit greatly
- Automatic adaptation

✅ **Mathematical Foundation**
- Accumulate squared gradients: $G_t = G_{t-1} + g_t^2$
- Adaptive update: $\theta_t = \theta_{t-1} - \frac{\alpha}{\sqrt{G_t + \epsilon}} g_t$
- Element-wise operations

✅ **Advantages**
- No manual learning rate tuning per parameter
- Excellent for sparse data (NLP, recommender systems)
- Fast initial convergence

✅ **Limitations**
- Diminishing learning rate problem
- May stop learning prematurely
- Not ideal for deep networks

### When to Use AdaGrad?

**Good For:**
- Sparse features (NLP, recommender systems)
- Convex optimization
- Short training runs

**Not Good For:**
- Deep neural networks
- Long training runs
- Non-convex optimization

### Connection to Other Notebooks

This notebook builds on:
- **`7_1-7_4`**: Previous optimizer notebooks

### Next Steps

🚀 **Coming Next:**
- **7.6 RMSProp**: Fixes AdaGrad's diminishing LR problem
- **7.7 Adam**: Combines momentum + RMSProp (most popular!)

---

**🎓 Congratulations!** You now understand AdaGrad and its adaptive learning rates!

**Key Insight:** AdaGrad adapts learning rates per parameter, but watch out for the diminishing learning rate problem!